# Workforce Scheduling with Skills and Coverage Constraints

Workforce scheduling is a common operational challenge in many
organizations, where a limited number of workers must be assigned to
work shifts while satisfying operational requirements.

Scheduling decisions are often complicated by factors such as skill
requirements, worker availability, and the need to balance workload
fairly across the workforce. As the number of workers and shifts grows,
manually constructing fair and feasible schedules becomes
increasingly difficult.

This notebook presents a workforce scheduling problem formulated and
solved using optimization techniques.


## Business Problem Statement

The goal of workforce scheduling is to assign workers to shifts in a way
that satisfies all operational requirements while producing a
reasonable and balanced schedule.

Given:
- A set of workers with specific skills and availability
- A set of shifts that must be covered
- Staff requirements specifying how many workers are needed per shift
- Skill requirements specifying which skills must be present in each shift.

The decision maker must determine which workers should be assigned to
which shifts. Each assignment decision is binary: a worker is either assigned to a
shift or not. Fractional assignments are not meaningful in this context.

A schedule is considered **valid** if:
- Every shifts are covered
- Skill requirements are satisfied
- Workers are only assigned to shifts for which they are available
- No worker is assigned to more than one shift.

Among all feasible schedules, some are preferable to others based on
criteria such as fairness and worker shift preferences.



## Modeling Overview

This workforce scheduling problem is modeled as a binary optimization
problem and formulated as a Mixed Integer Linear Program (MILP). Binary decision variables indicate whether a particular worker
is assigned to a particular shift.

The model includes:
- **Staff constraints** to ensure that each shift has sufficient
  staffing
- **Skill constraints** to ensure that required skills are present in
  each shift
- **Availability constraints** to prevent assigning workers to shifts
  they cannot work
- **Assignment constraints** to avoid overlapping or excessive
  assignments

In addition to these hard feasibility constraints, the model will later
incorporate soft constraints using penalty terms to improve schedule
quality, such as balancing workload or respecting worker preferences on shifts.


## Mathematical Formulation

This section presents the mathematical formulation of the
workforce scheduling problem. The formulation translates the business
requirements into decision variables, constraints, and objective
function suitable for an optimization problem.

#### Sets
\begin{align}
\mathcal{W} &: \text{Set of workers, }w\in\mathcal{W}\\
\mathcal{S} &: \text{Set of shifts, }s\in\mathcal{S}\\
\mathcal{K} &: \text{Set of skills, }k\in\mathcal{K}
\end{align}

#### Parameters
\begin{align}
\text{Availability parameter, }&\\a_{ws} &= \begin{cases}
1, ~& \text{if worker }w \text{ is available for the shift }s\\
0 , ~ & \text{otherwise}
\end{cases}\\
\text{Skill possession parameter, }&\\b_{wk} &= \begin{cases}
1 ,~& \text{if worker }w \text{ has skill }k\\
0 ,~ & \text{otherwise}
\end{cases}\\
\text{Skill requirement parameter, }&\\q_{sk} &= \begin{cases}
1 ,~& \text{if shift }s \text{ require the skill }k\\
0,~ & \text{otherwise}
\end{cases}\\
n_s &: \text{Required number of workers for the shift }s
\end{align}

#### Decision Variables
\begin{equation}
x_{ws} = \begin{cases}
1,~ &\text{if worker }w \text{ is assigned to the shift }s\\
0,~ &\text{otherwise}
\end{cases}
\end{equation}
**Interpretation:** It indicates whether a worker is assigned to a given shift.

#### Objective function
\begin{equation}
\min\sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}}~x_{ws}
\end{equation}
**Interpretation:** Although the primary purpose of the model is to ensure feasibility, this objective term plays an important role by preventing unnecessary overstaffing and guiding the solver toward solutions that use the minimum number of worker assignments required to satisfy all operational constraints.

#### Constraints
1. Staff Requirement Constraints: \begin{equation}\sum_{w\in\mathcal{W}} x_{ws} \geq n_s, \enspace \forall s\in\mathcal{S}\end{equation} **Interpretation:** Each shift must have at least the required number of workers.
2. Availability Constraints: \begin{equation} x_{ws} \leq a_{ws},\enspace\forall w\in\mathcal{W},\forall s\in\mathcal{S} \end{equation} **Interpretation:** Worker can only be assigned to shifts, for which they are available for.
3. Skill Requirement Constraints: \begin{equation} \sum_{w\in\mathcal{W}} x_{ws}b_{wk} \geq q_{sk},\enspace\forall s\in\mathcal{S}, \forall k\in\mathcal{K} \end{equation}  **Interpretation:** For each shift and each required skill, at least one of the assigned workers must possess that skill.
4. Workload Balancing Constraints: \begin{equation}\sum_{s\in\mathcal{S}} x_{ws} \leq1 ,\enspace\forall w\in\mathcal{W}.\end{equation} **Interpretation:** Each worker can be assigned to at most one shift.
5. Binary Constraints: \begin{equation} x_{ws}\in\{0,1\},\enspace \forall w\in\mathcal{W}, \forall s\in\mathcal{S}\end{equation}**Interpretation:** The variables are binary, since assignments are discrete decisions.


The following sections describe the construction of a synthetic dataset
and the implementation of this formulation using Pyomo. The underlying
formulation remains unchanged as the problem is scaled or extended.


## Test Case Construction

A small dataset is constructed to illustrate the workforce
scheduling model. The dataset includes workers, shifts, and skills, along with staffing and
skill requirements, worker availability schedules, and information on
skill possession for each worker.
The data is designed to be feasible by construction while still
introducing realistic scheduling challenges related to skill
availability and shift coverage.


In [22]:
# Sets and Parameters

workers = ['W1', 'W2', 'W3', 'W4', 'W5','W6']
shifts = ['Morning', 'Afternoon', 'Evening', 'Night']
skills = ['Skill_A', 'Skill_B']

staff_requirements = {
    'Morning':2,
    'Afternoon':2,
    'Evening':1,
    'Night':1
}
skill_requirements = {
    ('Morning', 'Skill_A'):1, ('Morning', 'Skill_B'):0,
    ('Afternoon', 'Skill_A'):0, ('Afternoon', 'Skill_B'):1,
    ('Evening', 'Skill_A'):1, ('Evening', 'Skill_B'):0,
    ('Night', 'Skill_A'):0, ('Night', 'Skill_B'):1
}
availability_schedule = {
    ('W1','Morning'):1, ('W1','Afternoon'):1, ('W1','Evening'):0, ('W1','Night'):0,
    ('W2','Morning'):1, ('W2','Afternoon'):1, ('W2','Evening'):1, ('W2','Night'):0,
    ('W3','Morning'):0, ('W3','Afternoon'):1, ('W3','Evening'):1, ('W3','Night'):1,
    ('W4','Morning'):1, ('W4','Afternoon'):0, ('W4','Evening'):1, ('W4','Night'):0,
    ('W5','Morning'):0, ('W5','Afternoon'):1, ('W5','Evening'):0, ('W5','Night'):1,
    ('W6','Morning'):1, ('W6','Afternoon'):1, ('W6','Evening'):1, ('W6','Night'):1
}
skill_possession = {
    ('W1','Skill_A'):1, ('W1','Skill_B'):0,
    ('W2','Skill_A'):1, ('W2','Skill_B'):1,
    ('W3','Skill_A'):0, ('W3','Skill_B'):1,
    ('W4','Skill_A'):1, ('W4','Skill_B'):0,
    ('W5','Skill_A'):0, ('W5','Skill_B'):1,
    ('W6','Skill_A'):1, ('W6','Skill_B'):1
}


In [23]:
# Building the pyomo model of the workforce scheduling problem

import pyomo.environ as pyo

def build_workforce_scheduling_model(
    workers, shifts, skills, staff_requirements, skill_requirements,
    availability_schedule, skill_possession):

    model = pyo.ConcreteModel()

# Sets:
    model.W = pyo.Set(initialize = workers)
    model.S = pyo.Set(initialize = shifts)
    model.K = pyo.Set(initialize = skills)

# Decision variables:
    model.x = pyo.Var(model.W, model.S, domain=pyo.Binary)

# Objective Function:
    model.obj = pyo.Objective(
        expr = sum(model.x[w,s] for w in model.W for s in model.S),
        sense = pyo.minimize
    )
# Constraints:

# Constraint 1: Staff Requirement Constraints
    def staff_rule(m,s):
        return sum(m.x[w,s] for w in m.W) >= staff_requirements[s]
        
    model.staff_constraint = pyo.Constraint(model.S, rule = staff_rule)
    
# Constraint 2: Availability Constraints
    def avail_rule(m,w,s):
        return m.x[w,s] <= availability_schedule[(w,s)]
    
    model.availability_constraint = pyo.Constraint(model.W, model.S, rule = avail_rule)
    
# Constraint 3: Skill Requirement Constraints
    def skill_rule(m,s,k):
        return sum(m.x[w,s] * skill_possession[(w,k)] for w in m.W) >= skill_requirements[(s,k)]

    model.skill_constraint = pyo.Constraint(model.S, model.K, rule = skill_rule)
    
# Constraint 4: Workload Balancing Constraints
    def work_rule(m,w):
        return sum(m.x[w,s] for s in m.S) <= 1

    model.workbalance_constraint = pyo.Constraint(model.W, rule = work_rule)

    return model


In [24]:
# A function to solve the workforce scheduling problem with the given solver

def solve_workforce_scheduling(model, solverName):
    solver = pyo.SolverFactory(solverName)
    results = solver.solve(model, tee = True)
    return results

In [25]:
# To print the Worker-Shift Schedule

import pandas as pd

def print_shift_schedule(model, title = "Duty Chart Schedule"):
    duty = []
    for w in model.W:
        for s in model.S:
            if pyo.value(model.x[w,s]) > 0.5:
                duty.append((w,s))

    line = "="*len(title)
    print(f"\n{line}\n{title}\n{line}")

    duty_df = pd.DataFrame(duty, columns = ['Worker','Shift'])
    return duty_df
    

In [26]:
# Solving the workforce scheduling problem by calling all the defined functions

solverName = 'cbc'
model = build_workforce_scheduling_model(
    workers, shifts, skills, staff_requirements, skill_requirements,
    availability_schedule, skill_possession)
results = solve_workforce_scheduling(model, solverName)

print("Solver status:", results.solver.termination_condition)
print(f"Optimal value : {pyo.value(model.obj)}")

duty_chart = print_shift_schedule(model,title = "Duty Chart Schedule")
duty_chart

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpa7by3o6u.pyomo.lp -stat=1 -solve -solu /tmp/tmpa7by3o6u.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 12 (-30) rows, 16 (-8) columns and 39 (-65) elements
Statistics for presolved model
Original problem has 24 integers (24 of which binary)
Presolved problem has 16 integers (16 of which binary)
==== 0 zero objective 1 different
16 variables have objective of 1
==== absolute objective values 1 different
16 variables have objective of 1
==== for integers 0 zero objective 1 different
16 variables have objective of 1
==== for integers absolute objective values 1 different
16 variables have objective of 1
===== end objective counts


Problem has 12 rows, 16 columns (16 with objective) and 39 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type f

,Worker,Shift
0,W1,Afternoon
1,W2,Morning
2,W3,Afternoon
3,W4,Morning
4,W5,Night
5,W6,Evening


### Interpretation of the Illustrative Example

The table above presents a feasible workforce schedule obtained by solving
the baseline optimization model using CBC. The schedule satisfies all
operational requirements: each shift meets its staffing and skill
requirements, workers are assigned only to shifts for which they are
available, and no worker is assigned to more than one shift. The solution
uses the minimum number of worker assignments required to meet these
requirements. While this schedule is optimal with respect to the
baseline objective, alternative schedules with the same objective value
may also exist.




## Enhancing Schedule Quality Using Soft Constraints

Although the baseline model ensures feasibility, it does not distinguish
between feasible schedules in terms of qualitative considerations such
as worker preferences or fairness. In real-world workforce scheduling,
such factors play an important role but are typically not enforced as
strict requirements.

To address this, preference and fairness considerations are introduced
as **soft constraints** through penalty terms in the objective function.
These penalties do not alter the feasible region of the model; instead,
they guide the optimizer toward higher-quality schedules by discouraging
undesirable assignments while preserving feasibility.

### Soft Constraints Considered in This Model

In this model, two types of soft constraints are incorporated:

- **Preference penalties**, which discourage assigning workers to shifts
  they find undesirable, while still allowing such assignments when
  necessary to maintain feasibility.

- **Fairness penalties**, which discourage uneven distribution of work
  across the workforce by penalizing schedules that unnecessarily burden
  the same workers when alternative feasible assignments exist.

These soft constraints are introduced solely through modifications to the
objective function. All hard constraints defining feasibility remain
unchanged.



### Modification to the Mathematical Formulation

The introduction of soft constraints does not alter the feasible region
defined by the original constraints. Instead, the objective function is extended to include penalty terms. Schematically, the objective function is modified from:
\begin{equation}
\min\sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}} x_{ws}
\end{equation}
to:
\begin{equation}
\min\enspace \sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}} x_{ws} + \alpha \sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}}P_{ws}~x_{ws} + \beta\sum_{w\in\mathcal{W}}Wl_w
\end{equation}
where
\begin{align}
P_{ws} &: \text{Penalty associated with assigning worker }w \text{ to the shift }s\\
\alpha, \beta &: \text{Weights to control the influence of preference and fairness penalty respectively}\\
Wl_w &: \text{Workload (total number of shifts) of the worker }w
\end{align}
and the workload is defined as
\begin{equation}
Wl_w = \sum_{s\in\mathcal{S}} x_{ws} \enspace\forall w\in\mathcal{W}.
\end{equation}

Finally, the objective function becomes:
\begin{equation}
\min~\sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}}x_{ws} + \underbrace{\alpha\sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}} P_{ws}~x_{ws}}_{\text{preference penalty term}} + \underbrace{\beta\sum_{w\in\mathcal{W}}\sum_{s\in\mathcal{S}}x_{ws}}_{\text{fairness penalty term}}.
\end{equation}


This formulation allows the optimizer to trade off schedule quality
against competing objectives while strictly maintaining feasibility.

In [27]:
# Preference data
## Note that 0 --> highly prefered
## larger values--> less prefered

preference = {
    ('W1', 'Morning'):0,
    ('W1', 'Afternoon'):1,
    ('W1', 'Evening'):1,
    ('W1', 'Night'):3,

    ('W2', 'Morning'):1,
    ('W2', 'Afternoon'):0,
    ('W2', 'Evening'):1,
    ('W2', 'Night'):2,

    ('W3', 'Morning'):3,
    ('W3', 'Afternoon'):1,
    ('W3', 'Evening'):0,
    ('W3', 'Night'):0,

    ('W4', 'Morning'):0,
    ('W4', 'Afternoon'):2,
    ('W4', 'Evening'):1,
    ('W4', 'Night'):3,

    ('W5', 'Morning'):3,
    ('W5', 'Afternoon'):0,
    ('W5', 'Evening'):2,
    ('W5', 'Night'):0,

    ('W6', 'Morning'):1,
    ('W6', 'Afternoon'):1,
    ('W6', 'Evening'):1,
    ('W6', 'Night'):1,
}
preference_weight = 1
fairness_weight = 0.5

In [28]:
# Build the Pyomo model with the soft constraints:

def build_workforce_scheduling_with_soft_constraints(
    workers, shifts, skills, staff_requirements, skill_requirements,
    availability_schedule, skill_possession, preference,
    preference_weight, fairness_weight):

    model = build_workforce_scheduling_model(
        workers, shifts, skills, staff_requirements, skill_requirements,
        availability_schedule, skill_possession)
    
    model.initial_objective = pyo.Expression(
        expr = sum(model.x[w,s] for w in model.W for s in model.S))

    model.preference_penalty = pyo.Expression(
        expr = sum(preference[(w,s)] * model.x[w,s]
        for w in model.W for s in model.S))
    
    model.fairness_penalty = pyo.Expression(
        expr = sum(model.x[w,s] for w in model.W for s in model.S))

# override the exisitng objective function
    model.obj.deactivate()

# add the objective function with penalty terms
    model.obj_modified = pyo.Objective(
        expr = model.initial_objective 
        + preference_weight * model.preference_penalty 
        + fairness_weight * model.fairness_penalty,
        sense = pyo.minimize
    )
    
    return model
    

In [29]:
# Solving the final workforce Scheduling problem with soft constraints:

model_soft = build_workforce_scheduling_with_soft_constraints(
    workers, shifts, skills, staff_requirements, skill_requirements,
    availability_schedule, skill_possession, preference,
    preference_weight, fairness_weight)

solver_name = 'cbc'
results_soft = solve_workforce_scheduling(model_soft, solver_name)
cbc_optimal = pyo.value(model_soft.obj_modified)

print("Solver status:", results_soft.solver.termination_condition)
print(f"Optimal value : {cbc_optimal}")

duty_chart_soft = print_shift_schedule(model_soft,title = "Duty Chart Schedule")
duty_chart_soft

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp__abrab2.pyomo.lp -stat=1 -solve -solu /tmp/tmp__abrab2.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 12 (-30) rows, 16 (-8) columns and 39 (-65) elements
Statistics for presolved model
Original problem has 24 integers (24 of which binary)
Presolved problem has 16 integers (16 of which binary)
==== 0 zero objective 2 different
7 variables have objective of 1.5
9 variables have objective of 2.5
==== absolute objective values 2 different
7 variables have objective of 1.5
9 variables have objective of 2.5
==== for integers 0 zero objective 2 different
7 variables have objective of 1.5
9 variables have objective of 2.5
==== for integers absolute objective values 2 different
7 variables have objective of 1.5
9 variables have objective of 2.5
===== end objective counts


Problem has 12 rows, 16 columns (1

,Worker,Shift
0,W1,Morning
1,W2,Afternoon
2,W3,Night
3,W4,Morning
4,W5,Afternoon
5,W6,Evening


### Schedule Visualization

The following tables present the workforce schedule obtained after solving
the optimization model with soft constraints. The schedule indicates which
workers are assigned to which shifts, while respecting all staffing,
skill, availability, and assignment constraints.

The visualization is provided in multiple views to support different
perspectives. A worker-centric view highlights individual assignments,
while a shift-centric view shows how staffing requirements are satisfied
for each shift. Together, these views allow for quick verification of
feasibility and intuitive assessment of schedule quality.


In [30]:
# worker-centric view

pivot_schedule = (
    duty_chart_soft
    .assign(Assigned=1)
    .pivot(index="Worker", columns="Shift", values="Assigned")
    .reindex(columns=shifts)
    .fillna("")
)

pivot_schedule


Shift,Morning,Afternoon,Evening,Night
Worker,,,,
W1,1.0,,,
W2,,1.0,,
W3,,,,1.0
W4,1.0,,,
W5,,1.0,,
W6,,,1.0,


In [31]:
# Shift-centric view

shift_view = (
    duty_chart_soft
    .groupby("Shift")["Worker"]
    .apply(list)
    .reindex(shifts)
)

shift_view


Shift
Morning      [W1, W4]
Afternoon    [W2, W5]
Evening          [W6]
Night            [W3]
Name: Worker, dtype: object

In [32]:
# Comparison of the duty schedule without and with soft constraints:

print("Baseline schedule:")
display(duty_chart)

print("Schedule with worker preferences:")
display(duty_chart_soft)


Baseline schedule:


,Worker,Shift
0,W1,Afternoon
1,W2,Morning
2,W3,Afternoon
3,W4,Morning
4,W5,Night
5,W6,Evening


Schedule with worker preferences:


,Worker,Shift
0,W1,Morning
1,W2,Afternoon
2,W3,Night
3,W4,Morning
4,W5,Afternoon
5,W6,Evening


### Interpretation: Schedule Quality Improvements Using Soft Constraints

The baseline model enforces all operational requirements through hard
constraints, ensuring feasibility with respect to staffing, skills,
availability, and assignment limits. To improve schedule quality, soft
constraints are incorporated as penalty terms in the objective function,
allowing the optimizer to favor preferred worker-shift assignments while
preserving feasibility.

All hard constraints remain strictly enforced, while soft constraints
guide the selection of higher-quality solutions within the feasible
region. This approach reflects common practice in real-world workforce
scheduling, where operational requirements are mandatory and qualitative
considerations are handled through controlled trade-offs.
.


## Solver Comparison: CBC vs CPLEX (Community Edition)

To assess solver portability and understand solver behavior on binary
optimization problems, the final workforce scheduling model (model with soft
constraints) is solved using both CBC and CPLEX (Community Edition).

#### CPLEX Availability Check

Before solving the model using CPLEX, solver availability is verified.


In [33]:
pyo.SolverFactory('cplex').available()

True

In [34]:
# Solving the final workforce scheluding model  with cplex solver

model_cplex = build_workforce_scheduling_with_soft_constraints(
    workers, shifts, skills, staff_requirements, skill_requirements,
    availability_schedule, skill_possession, preference,
    preference_weight, fairness_weight)

solver_Name = 'cplex'
results_cplex = solve_workforce_scheduling(model_cplex, solver_Name)
cplex_optimal = pyo.value(model_cplex.obj_modified)
print("Solver status:", results_cplex.solver.termination_condition)
print(f"Optimal value : {cplex_optimal}")

duty_chart_cplex = print_shift_schedule(model_cplex,title = "Duty Chart Schedule")
duty_chart_cplex


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer Community Edition 22.1.2.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2024.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmp9c3p_bi6.cplex.log' open.
CPLEX> Problem '/tmp/tmp2slxfa3i.pyomo.lp' read.
Read time = 0.01 sec. (0.00 ticks)
CPLEX> Problem name         : /tmp/tmp2slxfa3i.pyomo.lp
Objective sense      : Minimize
Variables            :      24  [Binary: 24]
Objective nonzeros   :      24
Linear constraints   :      42  [Less: 30,  Greater: 12]
  Nonzeros           :     104
  RHS nonzeros       :      30

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   : 1.500000         Max   : 4.500000       
Linear constraints   :
  Nonzeros   

,Worker,Shift
0,W1,Morning
1,W2,Afternoon
2,W3,Night
3,W4,Morning
4,W5,Afternoon
5,W6,Evening


## Solver Comparison Results and Interpretation


In [35]:
import pandas as pd

solver_comparison = pd.DataFrame({
    "Solver": ["CBC", "CPLEX"],
    "Optimal Value": [cbc_optimal, cplex_optimal]
})

solver_comparison.style.hide(axis='index').format({"Optimal Value":"{:.2f}"})


Solver,Optimal Value
CBC,10.00
CPLEX,10.00



Both CBC and CPLEX return the same optimal objective value for the
workforce scheduling problem with soft constraints. This confirms that
the formulation is solver-independent and correctly models the 
binary nature of the optimization problem.
While solver performance characteristics may differ for larger or more
complex instances, the consistency of results demonstrates that the
model can be reliably solved using both open-source and commercial
optimization solvers.


## Scalability Considerations and Key Takeaways

The workforce scheduling problem considered in this notebook is formulated
as a binary MILP. While the mathematical
formulation itself is general and can be applied to larger problem
instances without modification, workforce scheduling problems are
inherently combinatorial in nature. As problem size increases, solver
performance becomes strongly influenced by branching strategies,
problem structure, and the use of decomposition or heuristic techniques.

For this reason, a large-scale numerical experiment is intentionally
omitted in this project. The focus is placed instead on correct and
transparent modeling of feasibility requirements and schedule quality
through the careful separation of hard and soft constraints. In
practice, large-scale workforce scheduling problems are typically solved
using decomposition methods, rolling-horizon approaches, or hybrid
optimization-heuristic frameworks, which are beyond the scope of this
illustrative study.

### Key Takeaways
- Workforce scheduling problems can be effectively modeled as binary
  optimization problems with clearly defined feasibility constraints.
- Hard constraints enforce operational requirements such as staffing,
  skill coverage, availability, and assignment limits.
- Soft constraints can be incorporated as penalty terms in the objective
  function to improve schedule quality without sacrificing feasibility.
- The resulting formulation is solver-independent and can be solved
  using both open-source and commercial optimization solvers.
- Meaningful scaling of workforce scheduling models requires advanced
  algorithmic strategies rather than naive enlargement of problem size.

This project follows a modeling-first approach aligned with real-world
workforce scheduling practices and serves as a foundation for more
advanced extensions.
